In [1]:
import mysql.connector as sql
import pandas as pd
from mysql.connector import Error

## Problem Statement 18:

In [2]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = sql.connect(host=host_name, user=user_name, passwd=user_password)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"The error '{err}' occurred")

    return connection

pw = "Pratham123***"
connection = create_server_connection("localhost", "root", pw)

MySQL Database connection successful


In [3]:
def create_db(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"The error '{err}' occurred")


connection = create_db(connection, "CREATE DATABASE if not exists healthcare_db")

Database created successfully


In [4]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = sql.connect(host=host_name, user=user_name, passwd=user_password, database=db_name)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"The error '{err}' occurred")

    return connection

connection = create_db_connection("localhost", "root", pw, "healthcare_db")

MySQL Database connection successful


In [5]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"The error '{err}' occurred")

In [6]:
def run_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        df = pd.DataFrame(result, columns=columns)
        return df
    except Error as err:
        print(f"The error '{err}' occurred")

In [7]:
execute_query(connection, "USE healthcare_db")

Query successful


In [109]:
execute_query(connection, """CREATE TABLE IF NOT EXISTS census (
    District VARCHAR(255),
    State/UT VARCHAR(255),
    Population INT,
    Male BIGINT,
    Female INT,
    Literate INT,
    Literate_Male INT,
    Literate_Female INT,
    Households_Rural INT,
    Households_Urban INT,
    Households INT,
    Young_and_Adult INT,
    Middle_Aged INT,
    Senior_Citizen INT,
    Age_Not_Stated INT,
    PRIMARY KEY (District)
);""")


In [110]:
execute_query(connection, """CREATE TABLE IF NOT EXISTS housing(
District VARCHAR(255),
    Households_Rural INT,
    Households_Rural_Livable INT,
    Households_Rural_Dilapidated INT,
    Households_Rural_Toilet_Premise INT,
    Households_Urban INT,
    Households_Urban_Livable INT,
    Households_Urban_Dilapidated INT,
    Households_Urban_Toilet_Premise INT,
    PRIMARY KEY (District),
    FOREIGN KEY (District) REFERENCES census(District)
);
""")

In [111]:
execute_query(connection, """
              CREATE TABLE IF NOT EXISTS all_hospitals(
    State/UT VARCHAR(255),
    Number_of_Primary_Health_Centers_PHCs INT,
    Community_Health_Centers_CHCs INT,
    Sub_District_Divisional_Hospitals_SDHs INT,
    District_Hospitals_DHs INT,
    Hospitals INT,
    HospitalBeds INT,
    PRIMARY KEY (State/UT)
);
""")

In [112]:
execute_query(connection,""" CREATE TABLE IF NOT EXISTS government_hospital (
    State/UT VARCHAR(255),
    Rural_government_hospital INT,
    Rural_Government_Beds INT,
    Urban_government_hospital INT,
    Urban_Government_Beds INT,
    Last_Updated DATE,
    PRIMARY KEY (State/UT),
    FOREIGN KEY (State/UT) REFERENCES all_hospitals(State/UT)
);""")

## Problem Statement 19: 

In [8]:
run_query(connection, """SELECT *
FROM census
INNER JOIN housing ON census.`District Name`= housing.District
INNER JOIN all_hospitals ON census.`State/UT` = all_hospitals.`State/UT`
INNER JOIN government_hospital ON all_hospitals.`State/UT` = government_hospital.`State/UT`
LIMIT 3""")

,State/UT,District Name,Population,Male,Female,Literate,Literate_Male,Literate_Female,Households_Rural,Households_Urban,...,Sub-District/Divisional Hospitals(SDHs),District Hospitals(DHs),Hospitals,HospitalBeds,State/UT,Rural_Government_Hospitals,Rural_Government_Beds,Urban_Government_Hospitals,Urban_Government_Beds,Last_Updated
0,Himachal Pradesh,Chamba,519080,261320,257760,323842,186064,137778,186395,13836,...,61,15,671,8706,Himachal Pradesh,705,5665,96,6734,31-12-2017
1,Himachal Pradesh,Kangra,1510075,750591,759484,1152640,606443,546197,588321,39007,...,61,15,671,8706,Himachal Pradesh,705,5665,96,6734,31-12-2017
2,Himachal Pradesh,Kullu,437903,225452,212451,307672,174550,133122,167995,20709,...,61,15,671,8706,Himachal Pradesh,705,5665,96,6734,31-12-2017


## Problem Statement 20: 

## Problem Statement 21

In [9]:
execute_query(connection, """
              DROP FUNCTION IF EXISTS get_population_district;
              """)

Query successful


In [10]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS get_population_district(district_name TEXT)
              RETURNS INT
              DETERMINISTIC
              BEGIN
              DECLARE result INT;
              SELECT Population INTO result FROM census WHERE `District Name` = district_name;
              RETURN result;
              END;
              """)

Query successful


In [11]:
run_query(connection, "SELECT get_population_district('Ahmadabad')")

,get_population_district('Ahmadabad')
0,7214225


In [12]:
execute_query(connection, """
              DROP FUNCTION IF EXISTS get_population_state;
              """)

Query successful


In [13]:
execute_query(connection,"""
              CREATE FUNCTION IF NOT EXISTS get_population_state(state_name TEXT)
RETURNS INT
DETERMINISTIC
BEGIN
    DECLARE result INT;
    SELECT SUM(Population) INTO result
    FROM census
    WHERE `State/UT` = state_name
    GROUP BY `State/UT`;
    RETURN COALESCE(result, 0);
END;
              """)

Query successful


In [14]:
run_query(connection, "SELECT get_population_state('Gujarat');")

,get_population_state('Gujarat')
0,58404628


In [15]:
execute_query(connection, """
              DROP FUNCTION IF EXISTS senior_citizen_population;
              """)

Query successful


In [16]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS senior_citizen_population(state_name TEXT)
              RETURNS INT
              DETERMINISTIC
              BEGIN
              DECLARE result INT;
              SELECT SUM(Senior_Citizen) INTO result FROM census WHERE `State/UT` = state_name;
              RETURN result;
              END;
              """)

Query successful


In [17]:
run_query(connection, "SELECT senior_citizen_population('Gujarat');")

,senior_citizen_population('Gujarat')
0,9181111


In [18]:
execute_query(connection, """
              DROP FUNCTION IF EXISTS get_hospital_beds;
              """)

Query successful


In [19]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS get_hospital_beds(state_name TEXT)
              RETURNS INT
              DETERMINISTIC
              BEGIN
              DECLARE result INT;
              SELECT HospitalBeds INTO result FROM all_hospitals WHERE `State/UT` = `State_Name`;
              RETURN result;
              END;
              """)

Query successful


In [20]:
run_query(connection, "SELECT get_hospital_beds('Gujarat');")

,get_hospital_beds('Gujarat')
0,41129


In [21]:
execute_query(connection, "DROP FUNCTION IF EXISTS get_govt_hospital_beds;")

Query successful


In [22]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS get_govt_hospital_beds(state_name TEXT)
              RETURNS INT
              DETERMINISTIC
              BEGIN
              DECLARE result INT;
              SELECT Urban_Government_Beds+Rural_Government_Beds INTO result FROM government_hospital WHERE `State/UT` = state_name;
              RETURN result;
              END;
              """)

Query successful


In [23]:
run_query(connection, "SELECT get_govt_hospital_beds('Gujarat');")

,get_govt_hospital_beds('Gujarat')
0,32280


In [24]:
execute_query(connection, "DROP FUNCTION IF EXISTS beds_per_lakh;")

Query successful


In [25]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS beds_per_lakh(state_name TEXT)
              RETURNS FLOAT
              DETERMINISTIC
              BEGIN
              DECLARE result FLOAT;
              SELECT (get_hospital_beds(state_name)/(get_population_state(state_name))*100000) INTO result;
              RETURN result;
              END;
              """)

Query successful


In [26]:
run_query(connection, "SELECT beds_per_lakh('Gujarat');")

,beds_per_lakh('Gujarat')
0,70.4207


In [27]:
execute_query(connection, "DROP FUNCTION IF EXISTS govt_beds_per_lakh;")

Query successful


In [28]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS govt_beds_per_lakh(state_name TEXT)
              RETURNS FLOAT
              DETERMINISTIC
              BEGIN
              DECLARE result FLOAT;
              SELECT (get_govt_hospital_beds(state_name)/(get_population_state(state_name))*100000) INTO result;
              RETURN result;
              END;
              """)

Query successful


In [29]:
run_query(connection, "SELECT govt_beds_per_lakh('Gujarat');")

,govt_beds_per_lakh('Gujarat')
0,55.2695


In [30]:
execute_query(connection, "DROP FUNCTION IF EXISTS get_govt_hospital;")

Query successful


In [31]:
execute_query(connection, """
              CREATE FUNCTION IF NOT EXISTS get_govt_hospital(state_name TEXT)
              RETURNS INT
              DETERMINISTIC
              BEGIN
              DECLARE result INT;
              SELECT (Rural_Government_Hospitals+Urban_Government_Hospitals) INTO result FROM government_hospital WHERE `State/UT` = state_name;
              RETURN result;
              END;
              """)

Query successful


In [32]:
run_query(connection, "SELECT get_govt_hospital('Gujarat');")

,get_govt_hospital('Gujarat')
0,486


## Problem Statement 22:


In [33]:
with open('Data/north_east_states.txt', 'r') as f:
    north_east_states = f.read().splitlines()
north_east_states

['Arunachal Pradesh',
 'Assam',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Tripura ']

In [35]:
run_query(connection,"""
            SELECT `State/UT` AS State_Name, 
            get_population_state(`State/UT`) AS Population,
            senior_citizen_population(`State/UT`) AS `Senior Citizen Population`,
            get_govt_hospital(`State/UT`) AS `Number of Government Hospital`,
            get_govt_hospital_beds(`State/UT`) AS `Number of Government Hospital beds`,
            ROUND(govt_beds_per_lakh(`State/UT`)) AS `Number of Government Hospital Beds for 1L population`,
            ROUND(get_govt_hospital_beds(`State/UT`)/(senior_citizen_population(`State/UT`))*100000) as `Number of Government Hospital beds for 1L senior citizen`
            FROM census 
            WHERE `State/UT` IN ('Arunachal Pradesh','Assam','Manipur','Meghalaya','Mizoram','Nagaland','Tripura ')
            Group By `State/UT`
            ORDER BY govt_beds_per_lakh(`State/UT`) ASC
            """)

,State_Name,Population,Senior Citizen Population,Number of Government Hospital,Number of Government Hospital beds,Number of Government Hospital Beds for 1L population,Number of Government Hospital beds for 1L senior citizen
0,Manipur,2855794,409796,30,1427,50.0,348
1,Assam,31205576,4127732,1226,17142,55.0,415
2,Nagaland,1978502,211983,36,1880,95.0,887
3,Meghalaya,2966889,287754,157,4457,150.0,1549
4,Mizoram,1097206,144061,90,1997,182.0,1386
5,Arunachal Pradesh,1299780,131142,218,2404,185.0,1833


The error '1146 (42S02): Table 'healthcare_db.government_hospitals' doesn't exist' occurred


Query successful
